# Barotropic gyre

In [ ]:
using Oceananigans
using Oceananigans.Units 
using Printf

Nx = 256
Ny = 256

Lx = 1200kilometers # east-west extent [m]
Ly = 1200kilometers # north-south extent [m]
Lz = 5kilometers    # depth [m]

grid = RectilinearGrid(size = (Nx, Ny, 1),
                       x = (0, Lx),
                       y = (0, Ly),
                       z = (-Lz, 0),
                       topology = (Bounded, Bounded, Bounded))

β = 1.e-11
coriolis = BetaPlane(f₀=1e-4, β=β)

# Rβ = 1.e-4
# τ₀ = Rβ * β^2 * Lx^3 
τ₀ = 0.01 # 0.1 N/m^3 is realistic

# Forcing by zonal wind stress
@show surface_wind_stress_parameters = (τ₀ = τ₀, ρ₀ = 1000., Lφ = grid.Ly)
@inline surface_wind_stress(x, y, t, p) = -p.τ₀ / p.ρ₀ * sin(π * y / p.Lφ)
surface_wind_stress_bc = FluxBoundaryCondition(surface_wind_stress,
                                               parameters = surface_wind_stress_parameters)
u_bcs = FieldBoundaryConditions(top = surface_wind_stress_bc)

# Friction due to constant horizontal viscosity
@show νₕ = 200
constant_horizontal_diffusivity = HorizontalScalarDiffusivity(ν = νₕ)

@show model = HydrostaticFreeSurfaceModel(grid = grid,
                                    coriolis = coriolis,
                                    boundary_conditions = (u=u_bcs,),
                                    closure = constant_horizontal_diffusivity)

simulation = Simulation(model, Δt = 1200, stop_time = 1*365days)
output_fields = merge(model.velocities, (η=model.free_surface.η,))
output_prefix = "../data/raw_simulation_output/barotropic_gyre"

mutable struct Progress
    interval_start_time :: Float64
end

function (p::Progress)(sim)
    wall_time = (time_ns() - p.interval_start_time) * 1e-9

    @info @sprintf("Time: %s, iteration: %d, max(u): %.2e m s⁻¹, wall time: %s",
                   prettytime(sim.model.clock.time),
                   sim.model.clock.iteration,
                   maximum(sim.model.velocities.u),
                   prettytime(wall_time))

    p.interval_start_time = time_ns()

    return nothing
end

simulation.callbacks[:progress] = Callback(Progress(time_ns()), IterationInterval(5*3*24))


simulation.output_writers[:fields] = JLD2Writer(model, output_fields,
                                                schedule = TimeInterval(1days),
                                                filename = output_prefix,
                                                overwrite_existing = true)

@show run!(simulation)

surface_wind_stress_parameters = (τ₀ = τ₀, ρ₀ = 1000.0, Lφ = grid.Ly) = (τ₀ = 0.01, ρ₀ = 1000.0, Lφ = 1.2e6)
νₕ = 200 = 200
model = HydrostaticFreeSurfaceModel(grid = grid, coriolis = coriolis, boundary_conditions = (u = u_bcs,), closure = constant_horizontal_diffusivity) = HydrostaticFreeSurfaceModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 256×256×1 RectilinearGrid{Float64, Bounded, Bounded, Bounded} on CPU with 3×3×1 halo
├── timestepper: QuasiAdamsBashforth2TimeStepper
├── tracers: ()
├── closure: HorizontalScalarDiffusivity{ExplicitTimeDiscretization}(ν=200.0)
├── buoyancy: Nothing
├── free surface: ImplicitFreeSurface with gravitational acceleration 9.80665 m s⁻²
│   └── solver: FFTImplicitFreeSurfaceSolver
├── advection scheme: 
│   └── momentum: Vector Invariant, Dimension-by-dimension reconstruction
└── coriolis: BetaPlane{Float64}


In [ ]:
t = 365

using CairoMakie

η = FieldTimeSeries(string(output_prefix,".jld2"), "η")

x, y, z = nodes(η)
heatmap(x[:], y[:], interior(η)[:,:,1,t])